# Submission

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

In [5]:
# base feature set with advanced and mean encoded features
df_base_adv_mean = pd.read_csv('input/feature_sets/base_adv_mean.csv')

# reading the test file
features_to_drop = ['status_group','id']
df_test_X = pd.read_csv('input/feature_sets/base_adv_mean_test.csv')


# a little adjustment
df_test_X.funder_mean_nf.fillna(0, inplace=True) 
df_test_X.funder_mean_fr.fillna(0, inplace=True) 

######################################################################
###        RANDOM FOREST CLASSIFIER        ###########################
######################################################################

rf_balanced = RandomForestClassifier(n_jobs=-1, 
                                     verbose=1, 
                                     bootstrap=False, 
                                     max_depth=101,
                                     min_samples_leaf=1,
                                     min_samples_split=6,
                                     n_estimators=1000
                                    )



X = df_base_adv_mean.drop(['status_group','id','functional needs repair','non functional'], axis=1)
y = df_base_adv_mean['status_group'].values

rf_balanced.fit(X,y)

######################################################################


# prediction
rf_gs_best_mean_sub_1 = rf_balanced.predict(df_test_X.drop(features_to_drop, axis=1))

# preparing the file
df_test_X['status_group'] = rf_gs_best_mean_sub_1
df_gs_best_mean_sub_2 = df_test_X[['id','status_group']]

#saving the file
df_gs_best_mean_sub_2.to_csv('submissions/test_sub_gs_param_weight.csv', index=False)
del df_gs_best_mean_sub_2

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   39.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.1s finished
